In [1]:
import torch
from models import CNNRegressor, CNNClassifier, VGGRegressor
from dataloader import get_loaders

In [2]:
def get_model(model_type, seed):
    if model_type == "CNNRegressor":
        model = CNNRegressor()
        model_name = f"trained_models/cnnregressor_{seed}.pt"
    elif model_type == "CNNClassifier":
        model = CNNClassifier()
        model_name = f"trained_models/cnnlassifier_{seed}.pt"
    elif model_type == "VGGRegressor":
        model = VGGRegressor()
        model_name = f"trained_models/vggregressor_{seed}.pt"
    model.load_state_dict(torch.load(model_name))
    return model

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
train_fraction = 0.8
batch_size = 62
seed = 42
train_loader, val_loader = get_loaders(train_fraction, batch_size, seed)
model_type = "CNNRegressor"
model = get_model(model_type, seed).to(device)

In [9]:
if model_type == "CNNRegressor" or "VGGRegressor":
    ## Calculating overall accuracy of the validation set
    model.eval()
    total_len = 0
    total_correct = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            correct_count = (torch.sum(torch.round(outputs)==y)).item()
            total_len += len(y)
            total_correct += correct_count
    accuracy = (total_correct/total_len)*100
    print(f"Overall Accuracy of Regressor on Validation Split: {accuracy:.2f}%")

    total_len = 0
    total_correct = 0
    with torch.no_grad():
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            correct_count = (torch.sum(torch.round(outputs)==y)).item()
            total_len += len(y)
            total_correct += correct_count
    accuracy = (total_correct/total_len)*100
    print(f"Overall Accuracy of Regressor on Training Split: {accuracy:.2f}%")
else:
    model.eval()
    total_len = 0
    total_correct = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            outputs = F.softmax(outputs, dim=-1)
            outputs = torch.argmax(outputs, dim=1)
            correct_count = (torch.sum(torch.round(outputs)==y)).item()
            total_len += len(y)
            total_correct += correct_count
    accuracy = (total_correct/total_len)*100
    print(f"Overall Accuracy of Classifer on Validation Split: {accuracy:.2f}%")

    total_len = 0
    total_correct = 0
    with torch.no_grad():
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            outputs = F.softmax(outputs, dim=-1)
            outputs = torch.argmax(outputs, dim=1)
            correct_count = (torch.sum(torch.round(outputs)==y)).item()
            total_len += len(y)
            total_correct += correct_count
    accuracy = (total_correct/total_len)*100
    print(f"Overall Accuracy of Classifer on Training Split: {accuracy:.2f}%")

Overall Accuracy of Regressor on Validation Split: 10.65%
Overall Accuracy of Regressor on Training Split: 18.16%


The above part is just a copy of the relevant sections in the training code. Given the allround good performance of the CNN Regressor (seed 42) across both training and validation splits (without overfitting), that will be the final model I wish to submit for evaluation. In the subsequent code, I will train the model for a final time on the whole dataset (without train or test splitting) to ensure that the model is able to train over the complete data provided. The code can be commented if the final_model.pt file (representing the state dictionary of the model) is already present. 

In [9]:
from dataloader import get_loaders
from torch import nn
import torch.optim as optim
import numpy as np
import torch
from models import CNNRegressor

In [10]:
seed = 42
epochs = 15
learning_rate = 1e-3
batch_size = 64
device = "cuda" if torch.cuda.is_available() else "cpu"
train_loader = get_loaders(1, batch_size, seed, full_data_return= True)

In [11]:
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [12]:
final_model = CNNRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(final_model.parameters(), lr=learning_rate)

In [13]:
for epoch in range(epochs):
    final_model.train()
    train_loss = 0.0
    train_acc = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        accuracy = (torch.sum(torch.round(outputs)==labels)*100/len(labels)).item()
        train_acc += accuracy

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Average Train Accuracy: {train_acc/len(train_loader):.4f}")

Epoch 1/15, Train Loss: 32.7529, Average Train Accuracy: 6.6831
Epoch 2/15, Train Loss: 19.3948, Average Train Accuracy: 9.0307
Epoch 3/15, Train Loss: 14.0098, Average Train Accuracy: 10.6610
Epoch 4/15, Train Loss: 11.4056, Average Train Accuracy: 11.8925
Epoch 5/15, Train Loss: 9.8578, Average Train Accuracy: 13.0486
Epoch 6/15, Train Loss: 8.8819, Average Train Accuracy: 13.2507
Epoch 7/15, Train Loss: 8.2647, Average Train Accuracy: 14.3657
Epoch 8/15, Train Loss: 7.6257, Average Train Accuracy: 14.8943
Epoch 9/15, Train Loss: 7.3247, Average Train Accuracy: 14.8832
Epoch 10/15, Train Loss: 6.6234, Average Train Accuracy: 15.5983
Epoch 11/15, Train Loss: 6.0760, Average Train Accuracy: 16.6167
Epoch 12/15, Train Loss: 6.1472, Average Train Accuracy: 16.1270
Epoch 13/15, Train Loss: 4.8328, Average Train Accuracy: 18.0404
Epoch 14/15, Train Loss: 3.9300, Average Train Accuracy: 20.5413
Epoch 15/15, Train Loss: 4.0415, Average Train Accuracy: 20.1759


In [14]:
torch.save(final_model.state_dict(), f"./final_model.pt")

In [16]:
total_len = 0
total_correct = 0
with torch.no_grad():
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        outputs = final_model(x)
        correct_count = (torch.sum(torch.round(outputs)==y)).item()
        total_len += len(y)
        total_correct += correct_count
accuracy = (total_correct/total_len)*100
print(f"Overall Accuracy of Regressor on Training Split: {accuracy:.2f}%")

Overall Accuracy of Regressor on Training Split: 21.49%


As reported earlier in the training code, as well as the report, the validation accuracy of the current model is 10.65% but in different runs, it reaches up to 12-14% (owing to the stochastic nature of training models on GPU devices). The train accuracy reaches 20%. When no validation split is performed, the training accuracy on the full data reaches 21.49%. 